<a href="https://colab.research.google.com/github/k-madani/issues-summarizer/blob/master/notebooks/01_dataset_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>